# Chicago neighborhoods with a count of Food and Nightlife Spots

In [26]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

In [27]:
#Import the neighborhoods file from the html site
df = pd.read_html("https://www.seechicagorealestate.com/chicago-zip-codes-by-neighborhood.php")[0]

In [28]:
#Add additional columns
df['Latitude'] = ''
df['Longitude'] = ''
df['Restaurants'] = ''

#Add headers
my_columns = ["Neighborhood", "Zip", "Latitude", "Longitude", "Restaurants"]
df.columns = my_columns

df.shape

(174, 5)

In [29]:
#Drop areas outside of Chicago - all zips after North Shore
NorthShoreIndex = df[df['Neighborhood']=='North Shore'].index.item()
df = df[:NorthShoreIndex]

#Drop all nan rows
df = df.dropna()

df.shape

(81, 5)

In [30]:
#Drop rows that are category group headers
df = df[~df['Zip'].isin(['Zip Code'])]
df.reset_index(drop=True, inplace=True)

df.shape

(77, 5)

In [31]:
#Import additional libraries needed to get zip codes and display map

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [32]:
#Setup connection parameters to Foursquare
CLIENT_ID = 'NZ5LBK1TYKOTRJ5C0SEGHKRQACHJIUUERZZLZEMZXX1F5CMW' # your Foursquare ID
CLIENT_SECRET = 'IKS5HUYVHU5Y2P0S53MSOMJQ1EZH4TCXHOWQTMHYTNO0I05I' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NZ5LBK1TYKOTRJ5C0SEGHKRQACHJIUUERZZLZEMZXX1F5CMW
CLIENT_SECRET:IKS5HUYVHU5Y2P0S53MSOMJQ1EZH4TCXHOWQTMHYTNO0I05I


In [33]:
#Loop through df and set latitude and longitude
geolocator = Nominatim(user_agent="chicago_zips")

#foursquare values
LIMIT = 100 
radius = 500

for index, row in df.iterrows():
    zips = df.loc[index,'Zip']
    zipList = zips.split(", ")
    #print(zipList)
    
    latitudeTotal = 0
    longitudeTotal = 0
    counter = 0
    for neighborhoodZip in zipList:
        #print(neighborhoodZip)

        location = geolocator.geocode(neighborhoodZip)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of zip {} in {} are {}, {}.'.format(neighborhoodZip, df.loc[index,'Neighborhood'],latitude, longitude))

        counter = counter + 1
        latitudeTotal = latitudeTotal + latitude
        longitudeTotal = longitudeTotal + longitude

    latitudeTotal = latitudeTotal / counter
    longitudeTotal = longitudeTotal / counter
    
    #Set the latitude and longitude in the df
    df.loc[index,'Latitude'] = latitudeTotal
    df.loc[index,'Longitude'] = longitudeTotal
    
    #Get the number of restaurants
    #Food id 4d4b7105d754a06374d81259
    #Nightlife Spot id 4d4b7105d754a06376d81259
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryid={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitudeTotal, 
        longitudeTotal, 
        radius, 
        LIMIT,
        '4d4b7105d754a06374d81259,4d4b7105d754a06376d81259')

    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    nearby_venues = json_normalize(venues) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.location.address', 'venue.location.city', 'venue.location.postalCode']
    if nearby_venues.last_valid_index():
        nearby_venues =nearby_venues.loc[:, filtered_columns]
        maxRows = nearby_venues.last_valid_index() + 1
        df.loc[index,'Restaurants'] = maxRows
    
df.head()

The geograpical coordinate of zip 60611 in Cathedral District are 41.9013755, -87.623854.
The geograpical coordinate of zip 60605 in Central Station are 41.8705159580842, -87.6275595307936.
The geograpical coordinate of zip 60605 in Dearborn Park are 41.8705159580842, -87.6275595307936.
The geograpical coordinate of zip 60610 in Gold Coast are 41.9038725257493, -87.6385957471562.
The geograpical coordinate of zip 60611 in Gold Coast are 41.9013755, -87.623854.
The geograpical coordinate of zip 60601 in Loop are 9.42030540980842, -84.1550425023166.
The geograpical coordinate of zip 60602 in Loop are 41.8828718344361, -87.6291188024761.
The geograpical coordinate of zip 60603 in Loop are 41.8811030181372, -87.6282743003733.
The geograpical coordinate of zip 60604 in Loop are 41.8784178587146, -87.627759574011.
The geograpical coordinate of zip 60605 in Loop are 41.8705159580842, -87.6275595307936.
The geograpical coordinate of zip 60606 in Loop are 41.8828394635735, -87.6365272935001.
Th

,Neighborhood,Zip,Latitude,Longitude,Restaurants
0,Cathedral District,60611,41.9014,-87.6239,100
1,Central Station,60605,41.8705,-87.6276,72
2,Dearborn Park,60605,41.8705,-87.6276,72
3,Gold Coast,"60610, 60611",41.9026,-87.6312,100
4,Loop,"60601, 60602, 60603, 60604, 60605, 60606, 6060...",37.8177,-87.1983,


In [41]:
#If no restaurants found for the neighborhood set the value to 0
df.loc[df['Restaurants'] == '','Restaurants'] = 0

#Sort by the restaurants so we can identify the top, middle, and bottom 3rds
df = df.sort_values(by=['Restaurants','Neighborhood'], ascending=False, na_position='first')
df.reset_index(drop=True, inplace=True)

df.head()

,Neighborhood,Zip,Latitude,Longitude,Restaurants
0,Streeterville,60611,41.9014,-87.6239,100
1,River North,"60611, 60654",41.8963,-87.6282,100
2,River East,60611,41.9014,-87.6239,100
3,North Halsted,"60613, 60657",41.9447,-87.6594,100
4,Magnificent Mile,60611,41.9014,-87.6239,100


In [42]:
# create map of Chicago using last latitude and longitude values
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=12)

In [44]:
#Get number of indexes
maxRows = df.last_valid_index()

#Add the neighborhoods to the map
for index, row in df.iterrows():
    neighborhood = row['Neighborhood']
    lat = row['Latitude']
    lng = row['Longitude']
    restaurants = row['Restaurants']
    label = '{}, {}'.format(neighborhood, restaurants )
    label = folium.Popup(label, parse_html=True)
    
    #Set the colors based on how many restaurants in the neighborhood
    colorIndicator = 'green'
    if index >= maxRows / 3:
        if index >= maxRows / 3 * 2:
            colorIndicator = 'red'
        else:
            colorIndicator = 'yellow'
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=colorIndicator,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago) 
map_chicago

## Results
Looks like the Gold Coast, River North, Cathedral District is a great location to live.